In [1]:
from utils.all import *
data = load_data('../annotated', batch_num=[5, 6, 7, 8, 9], preprocess=True)

Found users: {'ayush', 'vinayak', 'vishnesh', 'rachel', 'kelly', 'anton'}



### Export `new-wiki-3` Outputs

In [2]:
# import csv

# simp_eval_rekey = {
#     'Original Sentence': 'original',
#     'id': 'original_id'
# }

# # Need to paste original simpeval id from simp_eval/simpeval_2022.csv
# path = '../yao/all-outputs-new-wiki-3.csv'
# with open(path, encoding='utf-8') as f:
#     reader = csv.reader(f)
#     keys = next(reader)
#     keys = [simp_eval_rekey[k] if k in simp_eval_rekey.keys() else k for k in keys]
#     contents = [row for row in reader]
#     simp_eval = []
#     for sent in contents:
#         simp_eval += [{k: v for k, v in zip(keys, sent)}]

# # Extract T5 sents
# out = []
# for sent in data:
#     if '11B' in sent['system']:
#         out += [sent['original']]
# final = []
# for sent_orig in set(out):
#     next_out = []
#     ref = []
#     for sent in data:
#         if '11B' in sent['system'] and sent['original'] == sent_orig:
#             next_out += [sent['simplified']]
#         if 'Human' in sent['system'] and sent['original'] == sent_orig:
#             ref += [sent['simplified']]
#     next_out = list(set(next_out))
#     ref = list(set(ref))
#     assert len(next_out) == 1
#     next_out = next_out[0]
#     final += [{
#         'original_id': int([x['original_id'] for x in simp_eval if x['original'] == sent_orig][0]),
#         'original': sent_orig, 
#         'generation': next_out, 
#         'system': 'T5-11B'
#         # 'references': ref
#     }]
# print(len(final))

# # Write to CSV
# f = csv.writer(open("comet/new-wiki-3-outputs.csv", "w", newline='', encoding='utf-8'))
# f.writerow(list(final[0].keys()))
# for row in final:
#     f.writerow([v for k, v in row.items()])

### BLEU

In [3]:
import nltk
hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

def BLEUsent(hypothesis, references):
    ref = [x.split(' ') for x in references]
    hyp = hypothesis.split(' ')
    BLEUscore = nltk.translate.bleu_score.sentence_bleu(ref, hyp)
    return BLEUscore

0.4548019047027907


In [4]:
# I should use this library because it's standardized and does tokenization
# more accurately
import sacrebleu

### SARI

In [5]:
from utils.sari import *
fnamenorm   = "./turkcorpus/test.8turkers.tok.norm"
fnamesimp   = "./turkcorpus/test.8turkers.tok.simp"
fnameturk  = "./turkcorpus/test.8turkers.tok.turk."

ssent = "About 95 species are currently accepted ."
csent1 = "About 95 you now get in ."
csent2 = "About 95 species are now agreed ."
csent3 = "About 95 species are currently agreed ."
rsents = ["About 95 species are currently known .", "About 95 species are now accepted .", "95 species are now accepted ."]

print(SARIsent(ssent, csent1, rsents))
print(SARIsent(ssent, csent2, rsents))
print(SARIsent(ssent, csent3, rsents))
print(SARIsent(ssent, csent3, rsents, components=True))

0.2682782411698074
0.5889995423074248
0.5071608864657479
(0.7714826593972436, 0.75, 0.0, 0.5071608864657479)


###  BERTScore

In [6]:
import bert_score as bs
from utils.util import avg

def BERTSCOREsent(hypothesis, references):
    hypothesis_all = [hypothesis for _ in range(len(references))]
    _, _, F1 = bs.score(hypothesis_all, references, lang='en', verbose=True)
    return avg(F1.tolist())

print(BERTSCOREsent(csent1, rsents))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.53 seconds, 1.96 sentences/sec
0.87


### COMET

In [7]:
# # !pip install unbabel-comet
# import comet # Import order matters! If you import everything first, you will get a WinError 127 because of incompatible libraries
# comet_model_path = comet.download_model('wmt21-comet-mqm')
# comet_mqm = comet.load_from_checkpoint(comet_model_path)

In [8]:
# import torch
# if torch.cuda.is_available():
#     torch.cuda.device_count()
#     torch.cuda.current_device()
#     torch.cuda.device(0)
#     print(torch.cuda.get_device_name(0))
# else:
#     print("No CUDA :(")

In [9]:
# data = [
#     {
#         "src": "Dem Feuer konnte Einhalt geboten werden",
#         "mt": "The fire could be stopped",
#         "ref": "They were able to control the fire."
#     },
#     {
#         "src": "Schulen und Kindergärten wurden eröffnet.",
#         "mt": "Schools and kindergartens were open",
#         "ref": "Schools and kindergartens opened"
#     }
# ]
# model_output = comet_mqm.predict(data, batch_size=8, gpus=1)
# seg_scores, system_score = model_output.scores, model_output.system_score

In [10]:
# COMET is giving an ambiguous pickle error, likely a problem with imports
# You can paste this code into Google Collab and it'll run fine

### Toy Example

In [11]:
source = "On the fifth day of flight, November 20, 2022, at 19:09 UTC, the Orion spacecraft entered the Lunar sphere of influence, thus the Moon's gravitational force became stronger than Earth's relative to the spacecraft."
simplification = "On November 20, 2022, the Orion spacecraft started to be affected by the Moon's gravity more than Earth's. This happened at 19:09 UTC, after five days of flight."
references = [
    "On the fifth day of flight, the Orion spaceship entered the Lunar sphere of influence. This means that the Moon's gravitational force could influence the spaceship more than the Earth's.",
    "On November 20, 2022, at 19:09 UTC, the Orion spacecraft entered the Lunar sphere of influence. This meant that the Moon was pulling the spacecraft with more gravitational force than the Earth."
]

print(SARIsent(source, simplification, references))
print(BLEUsent(simplification, references))
print(BERTSCOREsent(simplification, references))

0.35050527719306973
0.17490733920105417


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 28.36 sentences/sec
0.93


### Calculate Scores

In [19]:
# Create a scores list with references and simplifications
scores = []
for orig in set([x['original'] for x in data]):
    sents = [sent for sent in data if sent['original'] == orig]

    human = [sent for sent in sents if 'Human' in sent['system']]
    systems = [sent for sent in sents if 'Human' not in sent['system']]

    if len(systems) == 0:
        continue

    for system in systems:
        references = list(set([sent['simplified'] for sent in human]))
        prediction = system['simplified']
        
        # if system['simpeval_scores'] is None:
        #     continue
        # simpeval_score = avg(system['simpeval_scores'])

        score = {
            'original': orig,
            'simplified': prediction,
            'references': references,
            'system': system['system'],
            'simpeval': simpeval_score,
        }

        scores += [score]

print(f"Calculating sensitivity with scores for {len(scores)} sentences")

Calculating sensitivity with scores for 818 sentences


In [20]:
# Add calculated scores
for score in scores:    
    original, prediction, references = score['original'], score['simplified'], score['references']
    
    # Calculate BLEU
    try:
        score['bleu'] = BLEUsent(prediction, references)
    except Exception:
        print("Skipping corrput sentence...")
        continue

    # Calculate BERTScore
    score['bertscore'] = BERTSCOREsent(prediction, references)

    # Calculate SARI
    score['sari_add'], score['sari_keep'], score['sari_del'], score['sari'] = SARIsent(original, prediction, references, components=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.09 seconds, 21.42 sentences/sec


In [21]:
# Write scoring setup to json to add COMET scores
with open('comet/scores.json', 'w') as f:
    json.dump(scores, f, indent=4)

### Add COMET Scores

In [22]:
# Add COMET scores
with open('comet/comet_results.json', 'r') as f:
    comet_results = json.load(f)

for score in scores:
    # Ensure both the system and sentence is the same
    aligned = [sent for sent in comet_results if 
        sent['original'] == score['original'] and
        sent['system'] == score['system']
    ]
    
    if len(aligned) == 0 or 'comet' not in aligned[0].keys():
        score['comet'] = 0
        continue
    
    comet_score = aligned[0]['comet']
    score['comet'] = comet_score